# DeepSeek Agent 实战：小红书爆款文案生成助手

本 Notebook 将指导您如何使用 DeepSeek LLM 构建一个能够生成小红书爆款文案的智能 Agent。我们将从需求拆解开始，逐步定义 Agent 的系统提示词 (System Prompt)、外部工具 (Tools)，并实现其核心的工作流程，最终生成符合小红书平台特点的文案。

In [2]:
from openai import OpenAI
import os



api_key = os.getenv("DEEPSEEK_API_KEY")
deepseek_url = "https://api.deepseek.com/v1"
client= OpenAI(
    api_key=api_key,
    base_url=deepseek_url
)

## 2.需求拆解与 Agent 任务规划
#### 用户痛点与核心需求：
* **效率低下：** 人工创作周期长，难以满足高频发布需求
* **创意瓶颈：** 难以持续产出新颖、吸引人的爆款创意。
* **趋势捕捉难：** 实时流行元素难以快速融入文案。
* **平台特性把握：** 小红书特有风格（标题、正文、标签、表情）难以精准复制。

#### “爆款”文案的特征：
* **强吸引力标题：** 制造好奇、痛点共鸣、利益点清晰。
* **沉浸式正文：** 真实体验分享、细节描述、情感共鸣。
* **精准且多样标签：** 热门话题、品牌词、产品词、垂直领域词。
* **生动表情符号：** 增强表达力，提升活泼感。
* **清晰的行动召唤 (CTA)。** Call-to-Action

#### Agent 任务规划：核心工作流
1. **用户指令接收：** 接收产品信息、主题、风格等。
2. **信息收集 (Web Search/DB Query)：** 实时搜索行业趋势、热门话题、竞品分析、产品卖点。
3. **内容构思与初稿生成 (LLM)：** 结合所有信息，撰写标题、正文、标签、表情符号。
4. **风格与格式优化 (LLM)：** 根据小红书平台特点和指定风格，对文案进行润色和结构调整。
5. **最终输出：** 呈现完整文案。

流程满足T-A-O : Thought-Action-Observation : 思考-行为-评估  <=====>  Re-Act( Reasoning-Action)


## 3. 爆款文案生成逻辑与 Prompt 设计

### 3.1 System Prompt (系统提示词) 
**(核心)**

System Prompt 是 Agent 的“大脑”和“行为准则”。它定义了 Agent 的角色、目标以及工作方式。我们将采用 `Thought-Action-Observation` (ReAct) 模式来引导 DeepSeek 的推理过程。

In [5]:
# 构建系统提示词(角色)  指出让LLM 采用 Thought-Action-Observation 模式进行推理和行动
# 系统提示词也为字典格式
SYSTEM_PROMPT={
    "role":"system",
    "content":""""
你是一个资深的爆款小红书文案专家、同时是一名资深内容运行、删除结合最新潮流和产品卖点，创作引人入胜。高互动、高转化的笔记文案。
你的任务是根据用户提供的产品和需求，生成包含标题、正文、相关标签和笔记符号的完整小红书笔记。
请始终采用 `Thought-Action-Observation`模式进行推理和行动。文案风格需要积极、活泼、阳光、真诚且富有感染力，容易让人共情。当完成任务后请以 json 格式直接输出最终文案： 格式如下：
```json
{
  "title": "小红书标题",
  "body": "小红书正文",
  "hashtags": ["#标签1", "#标签2", "#标签3", "#标签4", "#标签5"],
  "emojis": ["✨", "🔥", "💖"]
}
```
在生成文案之前，请务必先思考并收集足够的信息。
"""}

## 3.Tools(工具定义)
Agent 的`双手`由一系列可调用的工具组成，这些工具拓展了 LLM 能力。使起能够获得实时信息、查询数据库或执行特定操作。定义3个核心工具：
- `search_web`: 用于搜索互联网上实时信息。如最新趋势、用户评价等
- `query_product_database`:用户查询产品数据库，获取产品详细卖点和特点。【模拟  @todo：改造成可以运行的】
- `generate_emoji`:用户根据文案生成恰当的表情符号。【模拟 @todo:改造成可运行的】

In [7]:
def _generate_tools_dict(function_name:str,function_desc:str,parameters:dict,required_params:list) ->dict:
    """
    参数:
        name (str): 工具名称，如 "search_web"
        description (str): 工具描述
        parameters (dict): 参数定义，格式为 {"参数名": {"type": "类型", "description": "描述"}}
        required_params (list): 必填参数名列表，如 ["query"]
        其中 function_desc 【很重要】 用于LLM 识别是否该调用的关键参数
    
    返回:
        dict: 符合 DeepSeek 规范的单个工具字典
    """
    dict_format = {
        "type": "function",
        "function": {
            "name": function_name,
            "description": function_desc,
            "parameters": {
                "type": "object",
                "properties": parameters,
                "required": required_params
            }
        }
    }
    return dict_format

In [8]:
search_tool = _generate_tools_dict(
    function_name="search_web",
    function_desc="搜索互联网上的实时信息，用于获取最新新闻、流行趋势、用户评价、行业报告等。请确保搜索关键词精确，避免宽泛的查询。",
    parameters={
        "query": {
            "type": "string",
            "description": "要搜索的关键词或问题，例如'最新小红书美妆趋势'或'深海蓝藻保湿面膜 用户评价'"
        }
    },
    required_params=["query"]
)
emoji_tool = _generate_tools_dict(
    function_name="generate_emoji",
    function_desc="根据提供的文本内容，生成一组适合小红书风格的表情符号。",
    parameters={
        "context": {
            "type": "string",
            "description": "文案的关键内容或情感，例如'惊喜效果'、'补水保湿'"
        }
    },
    required_params=["context"]
)
db_tool = _generate_tools_dict(
    function_name = "query_product_database",
    function_desc="查询内部产品数据库，获取指定产品的详细卖点、成分、适用人群、使用方法等信息。",
    parameters={
        "product_name": {
            "type": "string",
            "description": "要查询的产品名称，例如'深海蓝藻保湿面膜'"
        }
    },
    required_params = ["product_name"]
)
TOOLS_DEFINITION = [search_tool,emoji_tool,db_tool]


In [9]:
import json
tools_json = json.dumps(TOOLS_DEFINITION,ensure_ascii=False,indent=2)
print(tools_json)

[
  {
    "type": "function",
    "function": {
      "name": "search_web",
      "description": "搜索互联网上的实时信息，用于获取最新新闻、流行趋势、用户评价、行业报告等。请确保搜索关键词精确，避免宽泛的查询。",
      "parameters": {
        "type": "object",
        "properties": {
          "query": {
            "type": "string",
            "description": "要搜索的关键词或问题，例如'最新小红书美妆趋势'或'深海蓝藻保湿面膜 用户评价'"
          }
        },
        "required": [
          "query"
        ]
      }
    }
  },
  {
    "type": "function",
    "function": {
      "name": "generate_emoji",
      "description": "根据提供的文本内容，生成一组适合小红书风格的表情符号。",
      "parameters": {
        "type": "object",
        "properties": {
          "context": {
            "type": "string",
            "description": "文案的关键内容或情感，例如'惊喜效果'、'补水保湿'"
          }
        },
        "required": [
          "context"
        ]
      }
    }
  },
  {
    "type": "function",
    "function": {
      "name": "query_product_database",
      "description": "查询内部产品数据库，获取指定产品的详细卖点、成分、适用人群、使用方法等信息。",
    

### 3.3 模拟工具实现
由于无法直接调用真实的外部 api(如google search 或内部产品数据库)。我们将创建一些模拟 (Mock) 工具函数来演示 Agent 的工作流程。在实际应用中，您需要将这些模拟函数替换为真实的 API 调用。

In [11]:
import random # 模拟生成表情
import time  # 模拟网络延迟

def mock_search_web(query:str) ->str:
    print(f"[Tool Call] 模拟搜索网页：{query}")
    time.sleep(1) # 模拟网络延迟
    if "小红书美妆趋势" in query:
        return "近期小红书美妆流行'多巴胺穿搭'、'早C晚A'护肤理念、'伪素颜'妆容，热门关键词有#氛围感、#抗老、#屏障修复。"
    elif "保湿面膜" in query:
        return "小红书保湿面膜热门话题：沙漠干皮救星、熬夜急救面膜、水光肌养成。用户痛点：卡粉、泛红、紧绷感。"
    elif "深海蓝藻保湿面膜" in query:
        return "关于深海蓝藻保湿面膜的用户评价：普遍反馈补水效果好，吸收快，对敏感肌友好。有用户提到价格略高，但效果值得。"
    else:
        return f"未找到关于 '{query}' 的特定信息，但市场反馈通常关注产品成分、功效和用户体验。"


def mock_query_product_database(product_name: str) -> str:
    """模拟查询产品数据库，返回预设的产品信息。"""
    print(f"[Tool Call] 模拟查询产品数据库：{product_name}")
    time.sleep(0.5) # 模拟数据库查询延迟
    if "深海蓝藻保湿面膜" in product_name:
        return "深海蓝藻保湿面膜：核心成分为深海蓝藻提取物，富含多糖和氨基酸，能深层补水、修护肌肤屏障、舒缓敏感泛红。质地清爽不粘腻，适合所有肤质，尤其适合干燥、敏感肌。规格：25ml*5片。"
    elif "美白精华" in product_name:
        return "美白精华：核心成分是烟酰胺和VC衍生物，主要功效是提亮肤色、淡化痘印、改善暗沉。质地轻薄易吸收，适合需要均匀肤色的人群。"
    else:
        return f"产品数据库中未找到关于 '{product_name}' 的详细信息。"


def mock_generate_emoji_list(context:str)->list:
    """模拟生成表情符号，根据上下文提供常见标间"""
    print(f"[Tool Call] 模拟生成表情符号，上下文：{context}")
    time.sleep(0.2) # 模拟生成延迟
    if "补水" in context or "水润" in context or "保湿" in context:
        return ["💦", "💧", "🌊", "✨"]
    elif "惊喜" in context or "哇塞" in context or "爱了" in context:
        return ["💖", "😍", "🤩", "💯"]
    elif "熬夜" in context or "疲惫" in context:
        return ["😭", "😮‍💨", "😴", "💡"]
    elif "好物" in context or "推荐" in context:
        return ["✅", "👍", "⭐", "🛍️"]
    else:
        return random.sample(["✨", "🔥", "💖", "💯", "🎉", "👍", "🤩", "💧", "🌿"], k=min(5, len(context.split())))
# 名称->函数 映射表   
# 执行方式 ：通过字典查找到函数后，直接加括号 () 调用。
available_tools={
    "search_web":mock_search_web,
    "generate_emoji":mock_generate_emoji_list,
    "query_product_database":mock_query_product_database
}

## 4. 实战：构建小红书文案生成 Agent
将`System Prompt` 工具定会和模拟函数工具整合起来，构建出能够自动执行的Deppseek Agent 工具留 。核心是`generate_redbook_note()` 函数。通过一个循环模拟 Agent 的`Thought-Action-Observation`过程

In [13]:
import re

def generate_redbook_note(product_name:str,tone_style:str="活泼甜美",max_iterations:int = 5)->str:
    """
    使用Deepseek Agent 生成小红书爆款文案
    :param product_name: 要生成的产品名称
    :param tone_style: 文案的语气和风格，如"活泼可爱"、"知性"、"搞怪"等
    :param max_iterations:  Agent 最大迭代次数
    :return: 生成爆款文案(JSON 格式字符串)
    """
    print(f"\n 启动小红书文案生成助手 产品:{product_name} ,  文案风格:{tone_style}")
    # json {{}}  表示 非转义字符 正常输出{}   eg: {{....}} 输出 {....}
    USER_PROMPT={
        "role":"user",
        "content":f"请为产品「{product_name}」生成一篇小红书爆款文案。要求：语气{tone_style}，包含标题、正文、至少5个相关标签和5个表情符号。请以完整的JSON格式输出，并确保JSON内容用markdown代码块包裹（例如：```json{{...}}```）。"
    }
    messages = [
        SYSTEM_PROMPT,
        USER_PROMPT
    ]
    iteration_count = 0
    deepseek_model  = "deepseek-chat"
    while iteration_count<max_iterations:
        iteration_count+=1
        print(f"-- Iteration {iteration_count} --")
        try:
            # 调用 Deepseek API，传入对话历史和工具定义
            response  = client.chat.completions.create(
                model = deepseek_model,
                messages = messages,
                tools=TOOLS_DEFINITION,
                tool_choice="auto"
            )
            response_message = response.choices[0].message

            # **ReAct 模式:处理工具调用**
            # 如果模型决定调用工具
            if response_message.tool_calls:
                print(f"Agent 决定调用工具 .....")
                #将工具调用信息添加到对话历史中 表示message 已经由模型调用了工具。 也就是添加工具调用上下文
                messages.append(response_message)
                tools_outputs = []
                # 遍历大模型要调用的所有已注册函数
                for tool_call in response_message.tool_calls:
                    function_name = tool_call.function.name
                    # 保证调用函数的参数为合法的 json 字符串
                    # 即便工具不要求使用参数  也需要传入空的字典！
                    function_args =  json.loads(tool_call.function.arguments) if tool_call.function.arguments else {}
                    print(f"Agent 调用工具：  调用工具(function) 名称为:{function_name}  对应 tools 的 参数为 {function_args}")

                    # 调用工具
                    if function_name in available_tools:
                        # 根据定义的名字找到对应函数对象
                        tool_function = available_tools[function_name]
                        # 名称-函数映射表 调用函数
                        tool_result = tool_function(**function_args)
                        print(f"Observation : 工具 {function_name} 返回结果{tool_result}")
                        # 拼接函数输出结果。供返回给大模型  模型可以根据调用 id 知道调用了什么工具
                        tools_output = {
                            "tool_call_id":tool_call.id,
                            "role":"tool",
                            "content":str(tool_result)
                        }
                        tools_outputs.append(tools_output)
                    else:
                        error_msg = f"未注册工具 {function_name} 被调用！"
                        print(error_msg)
                        tools_output= {
                            "tool_call_id":tool_call.id,
                            "role":"tool",
                            "content":error_msg
                        }
                        tools_outputs.append(tools_output)
                # 将模型调用函数生成结果拼接回消息体中
                messages.extend(tools_outputs)
            # 当模型直接返回/输出内容的时候 表示生成了最终答案(  一般来说是这样)
            # **ReAct 模式:处理最终结果**
            elif response_message.content:
                print(f"模型生成结果:{response_message.content}")
                # 由于 USER_PROMPT 要求生成文本为 json 的 markdown  提取生成的 json 内容
                # re.DOTALL 表示改变正则表达式点号(.)的默认匹配行为， 默认.号表示匹配除换行符（\n）以外 的任意字符
                # 启用 re.DOTALL 标志时"." 会匹配包括换行符（\n）在内的所有字符
                json_string_match =  re.search(r"```json\s*(\{.*\})\s*```",response_message.content,re.DOTALL)
                if json_string_match:
                    # 提取匹配生成内容
                    extracted_json_content = json_string_match.group(1)
                    try:
                        final_response = json.loads(extracted_json_content)
                        print(f"Agent:任务完成,生成解析最终的 JSON 文案！")
                        return json.dumps(final_response,ensure_ascii=False,indent=2)
                    except Exception as e:
                        print(f"Agent:提取到 json 但是解析失败:{e}")
                        print(f"尝试解析的 json 字符串为\n :{extracted_json_content}")
                        # 解析失败 继续对话
                        messages.append(response_message)
            else:
                print("Agent : 未知异常,可能需要更多交互！")

        except Exception as e:
            print(f"调用 DeepSeek API 时发生错误: {e}")
            break
    print("\n Agent 达到最大迭代次数或未能生成最终文案、请检查 Prompt或增加迭代次数。")
    return "未能生成最终文案！"


## 5. 实际测试与文案生成

In [15]:
# 测试案例1 : 深海蓝藻保湿面膜
product_name_1 = "深海蓝藻保湿面膜"
tone_style_1 = "活泼甜美"
result_1 = generate_redbook_note(product_name_1,tone_style_1)
print("\n--- 生成的文案 1 ---")
print(result_1)


 启动小红书文案生成助手 产品:深海蓝藻保湿面膜 ,  文案风格:活泼甜美
-- Iteration 1 --
Agent 决定调用工具 .....
Agent 调用工具：  调用工具(function) 名称为:query_product_database  对应 tools 的 参数为 {'product_name': '深海蓝藻保湿面膜'}
[Tool Call] 模拟查询产品数据库：深海蓝藻保湿面膜
Observation : 工具 query_product_database 返回结果深海蓝藻保湿面膜：核心成分为深海蓝藻提取物，富含多糖和氨基酸，能深层补水、修护肌肤屏障、舒缓敏感泛红。质地清爽不粘腻，适合所有肤质，尤其适合干燥、敏感肌。规格：25ml*5片。
-- Iteration 2 --
Agent 决定调用工具 .....
Agent 调用工具：  调用工具(function) 名称为:generate_emoji  对应 tools 的 参数为 {'context': '补水保湿、修护肌肤、适合敏感肌'}
[Tool Call] 模拟生成表情符号，上下文：补水保湿、修护肌肤、适合敏感肌
Observation : 工具 generate_emoji 返回结果['💦', '💧', '🌊', '✨']
-- Iteration 3 --
模型生成结果:```json
{
  "title": "💦深海蓝藻保湿面膜｜敏感肌也能用的补水神器！✨",
  "body": "姐妹们！我终于找到了敏感肌的救星——深海蓝藻保湿面膜！💧\n\n作为一个常年干燥泛红的敏感肌，每次敷面膜都小心翼翼，生怕刺激皮肤。但这款面膜真的让我惊艳了！🌊\n\n🌟核心成分是深海蓝藻提取物，富含多糖和氨基酸，不仅深层补水，还能修护肌肤屏障，舒缓泛红！敷完皮肤水嫩嫩的，像喝饱了水一样～\n\n💖质地清爽不粘腻，完全不会闷痘，敏感肌也能放心用！而且一盒5片，性价比超高！\n\n📌使用方法：洁面后敷15-20分钟，轻轻按摩吸收，后续护肤效果加倍哦！\n\n#敏感肌必备 #补水神器 #深海蓝藻面膜 #护肤推荐 #面膜测评",
  "hashtags": ["#敏感肌必备", "#补水神器", "#深海蓝藻面膜", "#护肤推荐", "#面膜测评"],
  "emojis": ["💦", "

In [16]:
# 测试案例 2: 美白精华
product_name_2 = "美白精华"
tone_style_2 = "知性温柔"
result_2 = generate_redbook_note(product_name_2, tone_style_2,10)

print("\n--- 生成的文案 2 ---")
print(result_2)


 启动小红书文案生成助手 产品:美白精华 ,  文案风格:知性温柔
-- Iteration 1 --
Agent 决定调用工具 .....
Agent 调用工具：  调用工具(function) 名称为:query_product_database  对应 tools 的 参数为 {'product_name': '美白精华'}
[Tool Call] 模拟查询产品数据库：美白精华
Observation : 工具 query_product_database 返回结果美白精华：核心成分是烟酰胺和VC衍生物，主要功效是提亮肤色、淡化痘印、改善暗沉。质地轻薄易吸收，适合需要均匀肤色的人群。
-- Iteration 2 --
Agent 决定调用工具 .....
Agent 调用工具：  调用工具(function) 名称为:generate_emoji  对应 tools 的 参数为 {'context': '美白精华提亮肤色'}
[Tool Call] 模拟生成表情符号，上下文：美白精华提亮肤色
Observation : 工具 generate_emoji 返回结果['🔥']
-- Iteration 3 --
Agent 决定调用工具 .....
Agent 调用工具：  调用工具(function) 名称为:generate_emoji  对应 tools 的 参数为 {'context': '提亮肤色、淡化痘印'}
[Tool Call] 模拟生成表情符号，上下文：提亮肤色、淡化痘印
Observation : 工具 generate_emoji 返回结果['💯']
-- Iteration 4 --
Agent 决定调用工具 .....
Agent 调用工具：  调用工具(function) 名称为:generate_emoji  对应 tools 的 参数为 {'context': '改善暗沉'}
[Tool Call] 模拟生成表情符号，上下文：改善暗沉
Observation : 工具 generate_emoji 返回结果['💧']
-- Iteration 5 --
Agent 决定调用工具 .....
Agent 调用工具：  调用工具(function) 名称为:generate_emoji  对应 tools 的 参数为 {'cont

### 格式化生成的小红书文案
格式化LLM生成的小红书文案，提取标签、正文、标题和表情符号，组合成一个易读的 markdown格式的文本
**工作方式：**

1. **解析 JSON**：使用 `json.loads()` 将输入的字符串转换为 Python 字典。如果解析失败，会返回一个错误信息。
2. **提取数据**：使用 `.get()` 方法从字典中安全地提取 `title`、`body` 和 `hashtags`。使用 `.get()` 的好处是，如果某个键不存在，它会返回一个默认值（例如 `None` 或空列表），而不是抛出 `KeyError`。
3. **构建 Markdown 标题**：将 `title` 格式化为 Markdown 的二级标题 (`## Title`)。
4. **处理正文**：直接使用 `body`。由于小红书正文中的换行很重要，我们保留它们。
5. **处理 Hashtags**：将 `hashtags` 列表中的每个标签用空格连接起来，形成一行。
6. **表情符号 (Emojis)**：在小红书的实际发布中，表情符号通常已经嵌入在标题和正文中了。这个函数没有单独列出它们，因为这通常不是最终发布格式的一部分。如果需要，可以取消注释相关代码来单独显示它们。
7. **返回结果**：返回拼接好的 Markdown 字符串，并使用 `.strip()` 去除可能存在于末尾的多余空白。

In [18]:
import json
def format_rednote_for_markdown(json_string:str)->str:
    """
    将 JSON 格式的小红书文案转换成markdown 格式，以便阅读和发布。
    :param json_string: 包含小红书文案的 JSON 字符串。 格式为 {"title": "...", "body": "...", "hashtags": [...], "emojis": [...]}
    :return:
    """
    try:
        data = json.loads(json_string)
    except Exception as e:
        return f"无法解析 json 字符串 异常为:{e} \n 原始字符串为：{json_string}"
    # 获取标题、内容、标签
    title = data.get("title", "无标题")
    body = data.get("body", "")
    hashtags = data.get("hashtags", [])
    # 表情符号通常已经融入标题和正文中，这里可以选择是否单独列出
    emojis = data.get("emojis", [])
    # 构建 markdown 文本
    # 标题构建为二级标题
    markdown_output = f"## {title} \n\n"
    # 构建正文部分 保留换行符
    markdown_content = f"{body} \n\n"
    markdown_emoji= ""
    markdown_hashtag = ""
    # 构建标签
    if hashtags:
        # 小红书标签通常以空格分隔
        markdown_hashtag =  " ".join(hashtags)

    # 如果需要,可以单独列出 emoji 表情，但是通常已经包含在标题和正文中
    if emojis:
        emoji_string =  "".join(emojis)
        markdown_emoji = emoji_string
    markdown_output+=markdown_content
    markdown_output+=markdown_hashtag
    markdown_output+=markdown_emoji
    # 去掉末尾多余的空白
    return markdown_output.strip()


In [19]:
# --- 示例使用 ---
# 假设这是 generate_rednote 函数的输出
generated_json_output = """
{
  "title": "✨ 28天逆袭冷白皮！这款美白精华让我告别暗沉痘印 🌟",
  "body": "姐妹们！我终于找到了我的本命美白精华！💖\\n\\n作为一个常年熬夜➕痘印困扰的混油皮，肤色暗沉一直是我的心头大患。直到遇见了这款美白精华，简直打开了新世界的大门！🤩\\n\\n🌟 核心成分：烟酰胺+VC衍生物，双管齐下，提亮肤色效果绝绝子！\\n💧 质地轻薄到爆炸，上脸秒吸收，完全不会黏腻，油皮姐妹放心冲！\\n🌿 用了28天，痘印肉眼可见变淡了，整张脸都透亮了起来，素颜也能打！\\n\\n使用方法也很简单：早晚洁面后，滴2-3滴在手心，轻轻按压上脸，后续再叠加保湿产品就OK啦～\\n\\n真心推荐给所有想要均匀肤色、告别暗沉的姐妹！入股不亏！💖",
  "hashtags": ["#美白精华", "#提亮肤色", "#淡化痘印", "#护肤好物", "#冷白皮"],
  "emojis": ["✨", "💖", "🤩", "💧", "🌿"]
}
"""

# 调用格式化函数
markdown_note = format_rednote_for_markdown(generated_json_output)

# 打印结果
print("--- 格式化后的小红书文案 (Markdown) ---")
print(markdown_note)

# --- 另一个例子，假设JSON解析失败 ---
invalid_json_output = "{'title': 'Test', 'body': 'This is not valid json'}" # 使用单引号，非法
markdown_error_note = format_rednote_for_markdown(invalid_json_output)
print("\n--- 格式化错误示例 ---")
print(markdown_error_note)


--- 格式化后的小红书文案 (Markdown) ---
## ✨ 28天逆袭冷白皮！这款美白精华让我告别暗沉痘印 🌟 

姐妹们！我终于找到了我的本命美白精华！💖

作为一个常年熬夜➕痘印困扰的混油皮，肤色暗沉一直是我的心头大患。直到遇见了这款美白精华，简直打开了新世界的大门！🤩

🌟 核心成分：烟酰胺+VC衍生物，双管齐下，提亮肤色效果绝绝子！
💧 质地轻薄到爆炸，上脸秒吸收，完全不会黏腻，油皮姐妹放心冲！
🌿 用了28天，痘印肉眼可见变淡了，整张脸都透亮了起来，素颜也能打！

使用方法也很简单：早晚洁面后，滴2-3滴在手心，轻轻按压上脸，后续再叠加保湿产品就OK啦～

真心推荐给所有想要均匀肤色、告别暗沉的姐妹！入股不亏！💖 

#美白精华 #提亮肤色 #淡化痘印 #护肤好物 #冷白皮✨💖🤩💧🌿

--- 格式化错误示例 ---
无法解析 json 字符串 异常为:Expecting property name enclosed in double quotes: line 1 column 2 (char 1) 
 原始字符串为：{'title': 'Test', 'body': 'This is not valid json'}


### 6. 评估和优化
文案生成并非一蹴而就、需要持续的评估和优化。本节讨论一些评估方法和优化策略。
#### 评估文案的质量：
- 客观量化评估(数据):
    - 点赞/收藏/评论/分享：基础互动
    - 曝光/阅读/点击/涨粉：流量与曝光
    - 停留时长/截图率：用户行为
    - 商品页浏览/加购/ROI/成交转化:商业价值
    - 爆文率/同类横向对比:竞争对比
- 主观内部评论(人工):
    - 相关性：是否符合产品特点和主题
    - 吸引力：标题是否抓人，内容是否流畅
    - 合规性：是否敏感词、违规宣传
    - 风格匹配：是否符合小红书style和指定语气
    - 用户画像：目标人群的年龄、地域、兴趣标签

### 优化迭代方法
- Prompt 调整:根据评估结果、精修 SystemPrompt 、User Prompt ，增加会修改 Few Shot 示例。
- 工具拓展(tools 拓展)：引入新的工具(如敏感词检测、竞品分析工具)
- RAG(检索增强生成)：结合更精准的内部知识库、减少幻觉

## 7. 总结与展望

通过本次实战，我们成功构建了一个基于 DeepSeek Agent 的小红书爆款文案生成助手。我们学习了如何拆解需求、设计 Prompt、定义工具，并实现 Agent 的核心工作流。

Agent 在内容营销领域的潜力巨大，未来可以进一步拓展到：

*   **超个性化内容：** 根据用户数据，生成一对一的定制文案。
*   **多模态内容创作：** 结合图片、视频生成，实现图文音视频一体化。
*   **智能营销决策：** Agent 不仅生成内容，还能分析效果并给出投放建议。
*   **跨平台适配：** 快速生成适应不同社交媒体平台风格的文案。

同时，我们也需关注挑战，如确保内容真实性、处理高度主观情感、与现有工作流的无缝集成等。Agent 技术仍在快速发展，期待未来能带来更多惊喜！